## Final Exam Machine Learning

Davide Tateo; 167275

Francesca Salute; 167284

Nicole Favero; 167340 

Tomás Gonçalves; 167288

# Preparation 

In [1]:
#!git clone https://github.com/frasalute/ML_FinalExam.git

Cloning into 'ML_FinalExam'...
remote: Enumerating objects: 22346, done.
remote: Counting objects: 100% (8158/8158), done.
remote: Compressing objects: 100% (8143/8143), done.
remote: Total 22346 (delta 31), reused 8132 (delta 14), pack-reused 14188
Receiving objects: 100% (22346/22346), 401.91 MiB | 27.38 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Updating files: 100% (22257/22257), done.


In [1]:
import os
print(f"Current working directory: {os.getcwd()}")
print(f"Contents of the current directory: {os.listdir('.')}")

Current working directory: c:\Users\davit\Desktop\CBS\MACHINE LEARNING\GithubExam\ML_FinalExam
Contents of the current directory: ['.DS_Store', '.git', 'Final_Exam_Code.ipynb', 'images', 'LICENSE', 'README.md']


In [3]:
base_path = '/work/ML_FinalExam/images'
print("Base path is:", base_path)

Base path is: /kaggle/working/ML_FinalExam/images


In [ ]:
# pip install tensorflow
# pip install scikit-learn
# pip install seaborn
# pip install imagehash
# pip install keras-tuner

In [5]:
# Importing required API and modules
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import array_to_img
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import imagehash
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import time
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve, StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from kerastuner import RandomSearch
from kerastuner.tuners import RandomSearch

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2024-05-10 18:09:11.097988: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 18:09:11.098083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 18:09:11.217698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading the data - Resizing will be done after the filtering

In [6]:
def load_images(base_path, categories):
    data = {'photo_id': [], 'image': [], 'category': [], 'subset': []}
    
    for subset in ['train', 'test']:
        for category in categories.keys():
            folder_path = os.path.join(base_path, subset, category)
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.jpg', '.jpeg')):  # Check for both .jpg and .jpeg extensions
                    file_path = os.path.join(folder_path, filename)
                    # Load the image
                    image = load_img(file_path)
                    # Add data to the DataFrame
                    data['photo_id'].append(filename)
                    data['image'].append(image)
                    data['category'].append(categories[category])
                    data['subset'].append(subset)
    
    df = pd.DataFrame(data)
    return df

# Load the DataFrame
categories = {'Benign': 0, 'Malignant': 1, 'Undetected': 2}
base_path = '/work/ML_FinalExam/images'
df = load_images(base_path, categories)

In [7]:
print(df.head())
row_count = len(df)
print("Number of rows in the DataFrame:", row_count)

   photo_id                                              image  \
0  6154.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
1  3582.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
2  2657.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
3     2.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
4  4941.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   

                                         image_array  category  
0  [[[170.0, 129.0, 145.0], [172.0, 131.0, 147.0]...         0  
1  [[[140.0, 108.0, 87.0], [141.0, 109.0, 88.0], ...         0  
2  [[[170.0, 130.0, 156.0], [174.0, 134.0, 160.0]...         0  
3  [[[253.0, 202.0, 245.0], [255.0, 205.0, 248.0]...         0  
4  [[[182.0, 144.0, 155.0], [183.0, 145.0, 156.0]...         0  
Number of rows in the DataFrame: 13879


In [ ]:
# Category Distribution Bar Chart showing for Benign, Malignant and Undetected how many images belong to each

plt.figure(figsize=(10, 7))
sns.countplot(data=df, x='category')
plt.title('Category Distribution') # Presenting the different categories of the dataset and the number of images in those categories
plt.xlabel('Category')
plt.ylabel('Number of Images')
plt.xticks(ticks=[0, 1, 2], labels=['Benign', 'Malignant', 'Undetected'])  
plt.show()

In [ ]:
# Subset Distribution Bar Chart showing for Trial and Test how many images belong to each

plt.figure(figsize=(10, 7))
sns.countplot(data=df, x='subset')
plt.title('Subset Distribution') # Presenting the two subsets of the dataset and the number of images belonging to each subsets
plt.xlabel('Subset')
plt.ylabel('Number of Images')
plt.xticks(ticks=[0, 1], labels=['Train', 'Test'])  
plt.show()

# Augmentation of "undetected" datasets

In [8]:
# Set the source directory and the target directories
original_undetected_dir = './images/undetec_to_augm'
train_augm_undetected_dir = './images/train/Undetected'
test_augm_undetected_dir ='./images/test/Undetected'

# Create a data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range = [0.8, 1.3],
    fill_mode='nearest'
)

def augment_img_dataset(source_dir, target_dir, target_count, imagedata_generator):

    """
        #this function performs augmentation

        # source_dir:           directory containing the original dataset to augment
        # target_dir:           directory where to save the augmented images
        # target_count:         desired number of images after augmentation
        # imagedata_generator:  image data generator object to be used for aumentation

        # returns:              augmented set of images
    """ 

    # List the files in the source directory containing the original dataset 
    file_list = os.listdir(source_dir)
    # Number of images in the original dataset
    original_count = len(file_list)

    # Number of times to repeat each image to reach the target count
    repeats_1 = min(target_count // original_count + 1, len(file_list))

    # Ensure the target directories exist
    os.makedirs(target_dir, exist_ok=True)    

    # Augment and save the images for the 6000 train images
    for file in file_list:
        img_path = os.path.join(source_dir, file)
        img = load_img(img_path)
        img = img_to_array(img)
        img = img.reshape((1,) + img.shape)

        i = 0
        for batch in imagedata_generator.flow(img, batch_size=1, save_to_dir = target_dir, save_prefix='augm', save_format='jpeg'): 
            #if in the previous line i save them as jpg, it anyway augments only the jpeg ones
            i += 1
            if i >= repeats_1:
                break  # break the loop after reaching the desired number of augmented images

### AVOID RUNNING THE FOLLOWING IF DATASET HAS ALREADY BEEN AUGMENTED ###
            
# Perform augmentation to expand the train and test datasets for "undetected"
    #augment_img_dataset(original_undetected_dir, train_augm_undetected_dir, 6000, datagen)
    #augment_img_dataset(original_undetected_dir, test_augm_undetected_dir, 1000, datagen)

'original_dataset_dir = \'./images/undetec_to_augm\'\ntrain_augm_undetected_dir = \'./images/train/Undetected\'\ntest_augm_undetected_dir =\'./images/test/Undetected\'\n\n# Create a data generator for augmentation\ndatagen = ImageDataGenerator(\n    rotation_range=30,\n    width_shift_range=0.2,\n    height_shift_range=0.2,\n    shear_range=0.2,\n    zoom_range=0.2,\n    horizontal_flip=True,\n    brightness_range = [0.8, 1.3],\n    fill_mode=\'nearest\'\n)\n\n# List the files in the original dataset directory\nfile_list = os.listdir(original_dataset_dir)\n\n# Ensure the target directories exist\nos.makedirs(train_augm_undetected_dir, exist_ok=True)\nos.makedirs(test_augm_undetected_dir, exist_ok=True)\n\n# Desired number of images after augmentation\ntarget_count_1 = 6000\ntarget_count_2 = 1000\n"""= target_count_2 = target_count_3 = target_count_4 = target_count_5 = target_count_6 = target_count_7"""\n\n# Number of images in the original dataset\noriginal_count = len(file_list)\n\n\n

# Filtering

Checking for Duplicates

In [ ]:
# Compute different types of hashes for each image and add to DataFrame
def compute_hashes(df):
    a_hashes = []
    d_hashes = []
    w_hashes = []
    for image in df['image']:
        # No need to convert to array, directly use the PIL image object
        a_hashes.append(str(imagehash.average_hash(image)))
        d_hashes.append(str(imagehash.dhash(image)))
        w_hashes.append(str(imagehash.whash(image)))
    df['a_hash'] = a_hashes
    df['d_hash'] = d_hashes
    df['w_hash'] = w_hashes

compute_hashes(df)

# Check for duplicates based on different hashes
duplicate_a_hash = df[df.duplicated(subset='a_hash', keep=False)]
duplicate_d_hash = df[df.duplicated(subset='d_hash', keep=False)]
duplicate_w_hash = df[df.duplicated(subset='w_hash', keep=False)]

print("Duplicate Images based on Average Hash:")
print(duplicate_a_hash[['photo_id', 'category', 'subset', 'a_hash']])

print("Duplicate Images based on Difference Hash:")
print(duplicate_d_hash[['photo_id', 'category', 'subset', 'd_hash']])

print("Duplicate Images based on Wavelet Hash:")
print(duplicate_w_hash[['photo_id', 'category', 'subset', 'w_hash']])


In [ ]:
# Function to visualize duplicate images
def visualize_duplicates(df, hash_column):
    duplicates = df[df.duplicated(subset=hash_column, keep=False)]
    grouped = duplicates.groupby(hash_column)
    for hash_value, group in grouped:
        if len(group) > 1:
            fig, axes = plt.subplots(1, len(group), figsize=(15, 5))
            for ax, (_, row) in zip(axes, group.iterrows()):
                ax.imshow(row['image'])
                ax.set_title(f"{row['photo_id']}\n{row['category']}\n{row['subset']}")
                ax.axis('off')
            plt.suptitle(f"Duplicates based on {hash_column}: {hash_value}")
            plt.show()

# Visualize duplicates for each hash type
visualize_duplicates(df, 'a_hash')
visualize_duplicates(df, 'd_hash')
visualize_duplicates(df, 'w_hash')

In [ ]:
# Manually specify the photo_id values to remove
photo_ids_to_remove = [
    '62.jpg', '1075.jpg', '341.jpg', '217.jpg', '1460.jpg', '410.jpg', '4035.jpg', 
    '2265.jpg', '155.jpg', '305.jpg', '1108.jpg', '2288.jpg', '1193.jpg', '943.jpg', 
    '2266.jpg', '883.jpg', '362.jpg', '890.jpg', '1089.jpg', '1497.jpg', '3410.jpg', 
    '1172.jpg', '3255.jpg', '526.jpg', '286.jpg', '3190.jpg', '2083.jpg', '522.jpg', 
    '104.jpg', '2076.jpg', '384.jpg', '1458.jpg', '248.jpg', '415.jpg', '189.jpg', 
    '323.jpg', '112.jpg', '375.jpg', '178.jpg', '1316.jpg', '2096.jpg', '118.jpg', 
    '250.jpg', '49.jpg', '294.jpg', '417.jpg', '1153.jpg', '5366.jpg', '432.jpg', 
    '874.jpg', '71.jpg'
]

# Remove the specified duplicates from the DataFrame
df_cleaned = df[~df['photo_id'].isin(photo_ids_to_remove)]

# Count images per category before and after removal of duplicates
def count_images_per_category(df):
    return df['category'].value_counts()

print("Image count per category before removal:")
print(count_images_per_category(df))

print("Image count per category after removal:")
print(count_images_per_category(df_cleaned))

print(f"Original DataFrame size: {len(df)}")
print(f"Cleaned DataFrame size: {len(df_cleaned)}")

df = df_cleaned

Check the size of the images before resize them

In [ ]:
# Function to calculate the aspect ratio of an image
def aspect_ratio(image):
    width, height = image.size
    return width / height

# Function to filter images based on aspect ratio
def filter_aspect_ratio(df_cleaned):
    filtered_df = df_cleaned[df_cleaned.apply(lambda row: 0.25 <= aspect_ratio(row['image']) <= 4.0, axis=1)]
    return filtered_df

# Apply the aspect ratio filter to the DataFrame
df_aspect_filtered = filter_aspect_ratio(df_cleaned)

print(f"Original DataFrame size: {len(df_cleaned)}")
print(f"Aspect Ratio Filtered DataFrame size: {len(df_aspect_filtered)}")

Check the average RGB values

In [ ]:
# Function to calculate the average RGB values of an image
def average_rgb(image):
    np_image = np.array(image)
    avg_r = np.mean(np_image[:, :, 0])
    avg_g = np.mean(np_image[:, :, 1])
    avg_b = np.mean(np_image[:, :, 2])
    return avg_r, avg_g, avg_b

# Function to filter images based on average RGB values and return filtered out images
def filter_rgb_values(df):
    filtered_out_indices = []
    for idx, row in df.iterrows():
        avg_r, avg_g, avg_b = average_rgb(row['image'])
        if not (5 <= avg_r <= 250 and 5 <= avg_g <= 250 and 5 <= avg_b <= 250):
            filtered_out_indices.append(idx)
    filtered_out_df = df.loc[filtered_out_indices]
    filtered_df = df.drop(filtered_out_indices)
    return filtered_df, filtered_out_df

# Apply the RGB value filter to the aspect ratio filtered DataFrame
df_final_filtered, df_filtered_out = filter_rgb_values(df_aspect_filtered)

print(f"Aspect Ratio Filtered DataFrame size: {len(df_aspect_filtered)}")
print(f"Final Filtered DataFrame size: {len(df_final_filtered)}")
print(f"Filtered Out DataFrame size: {len(df_filtered_out)}")

# Function to visualize filtered out images
def visualize_filtered_images(df):
    for idx, row in df.iterrows():
        image = row['image']
        plt.imshow(image)
        plt.title(f"Photo ID: {row['photo_id']}\nCategory: {row['category']}\nSubset: {row['subset']}")
        plt.axis('off')
        plt.show()

# Visualize the filtered out images
visualize_filtered_images(df_filtered_out)

Resize the images to 128 x 128 pixel

In [ ]:
def resize_images(df, target_size=(128, 128)): 
    for index, row in df.iterrows():
        image = row['image']
        # Resize the image
        resized_image = image.resize(target_size)
        image_array = img_to_array(resized_image)
        # Overwrite data in the original DataFrame
        df.at[index, 'image'] = resized_image
        df.at[index, 'image_array'] = image_array
    
    return df

# Resize the images
df = resize_images(df)

In [ ]:
# Presenting the above graphs again to show differences between before and after filtering

# Category Distribution Bar Chart showing for Benign, Malignant and Undetected how many images belong to each 

plt.figure(figsize=(10, 7))
sns.countplot(data=df, x='category')
plt.title('Category Distribution') # Presenting the different categories of the dataset and the number of images in those categories
plt.xlabel('Category')
plt.ylabel('Number of Images')
plt.xticks(ticks=[0, 1, 2], labels=['Benign', 'Malignant', 'Undetected'])  
plt.show()

In [ ]:
# Subset Distribution Bar Chart showing for Trial and Test how many images belong to each

plt.figure(figsize=(10, 7))
sns.countplot(data=df, x='subset')
plt.title('Subset Distribution') # Presenting the two subsets of the dataset and the number of images belonging to each subsets
plt.xlabel('Subset')
plt.ylabel('Number of Images')
plt.xticks(ticks=[0, 1], labels=['Train', 'Test'])  
plt.show()

In [ ]:
# Color Matrix

# PCA

In [11]:
# Making sure it's a 2D array
image_matrix = np.stack(df['image_array'].values)
print(image_matrix)

[[[[170. 129. 145.]
   [172. 131. 147.]
   [170. 129. 145.]
   ...
   [202. 167. 174.]
   [200. 165. 172.]
   [199. 164. 171.]]

  [[169. 128. 144.]
   [171. 130. 146.]
   [172. 131. 147.]
   ...
   [202. 167. 174.]
   [200. 165. 172.]
   [199. 164. 171.]]

  [[170. 129. 145.]
   [171. 130. 146.]
   [173. 132. 146.]
   ...
   [202. 167. 174.]
   [201. 166. 173.]
   [199. 164. 171.]]

  ...

  [[176. 144. 149.]
   [179. 147. 152.]
   [183. 148. 154.]
   ...
   [195. 159. 163.]
   [193. 157. 161.]
   [191. 155. 159.]]

  [[174. 142. 147.]
   [177. 145. 150.]
   [182. 147. 153.]
   ...
   [196. 160. 164.]
   [193. 157. 161.]
   [191. 155. 159.]]

  [[173. 141. 146.]
   [176. 144. 149.]
   [181. 146. 152.]
   ...
   [195. 159. 163.]
   [193. 157. 161.]
   [190. 154. 158.]]]


 [[[140. 108.  87.]
   [141. 109.  88.]
   [143. 111.  90.]
   ...
   [133. 100.  85.]
   [147. 114.  99.]
   [137. 104.  89.]]

  [[140. 108.  87.]
   [141. 109.  88.]
   [143. 111.  90.]
   ...
   [134. 101.  86.]
 

In [ ]:
# It's a three dimensions array so we have to flatten it 
image_matrix = np.array([img.flatten() for img in df['image_array']])

# Scale the data
scaler = StandardScaler()
image_matrix_scaled = scaler.fit_transform(image_matrix)

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# Principal Component Analysis
pca_ratio=PCA()
pca_ratio.fit(image_matrix_scaled) # fit the PCA so it can learn

# Using cumulative variance
cumulative_variance_ratio=np.cumsum(pca_ratio.explained_variance_ratio_)
variance=0.95 # set to 95% to keep a sufficiently large portion of the variance
n_components= np.argmax(cumulative_variance_ratio >= variance) +1 # find the number of components needed 

print(f"Number of principal components: {n_components}")

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")

# There is another way to obtain the number of components which is setting the threshold 
# pca_threshold = PCA(n_components=0.95)
# pca_threshold.fit(image_matrix_scaled) - fit the pca model to the data to learn patterns
# print(f"Number of principal components: {pca_threshold.n_components_}")

In [ ]:
# Screen plot eigenvalues - number of principal components
plt.figure(figsize=(10, 6))
sns.lineplot(x=np.arange(1, len(cumulative_variance_ratio) + 1), y=cumulative_variance_ratio, marker='o', color='#FF69B4')
plt.title('Scree Plot')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Transform the original data using retained principal components 
# Start from inputting in PCA the number of components found necessary for 95% variance
pca_opt=PCA(n_components=99)
pca_opt.fit(image_matrix_scaled)

df_matrix_reduced= pca_opt.transform(image_matrix_scaled) # transform the flatten original data to reduced dimensionality

# SVM with PCA

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# Split the Data in test and train
train_indices = df[df['subset'] == 'train'].index
test_indices = df[df['subset'] == 'test'].index

X_train = df_matrix_reduced[train_indices]
X_test = df_matrix_reduced[test_indices]
y_train = df.loc[train_indices, 'category']
y_test = df.loc[test_indices, 'category']

# Definition of the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.0001, 0.001, 0.1, 1],
    'kernel': ['rbf', 'poly']
}

# Creation of an SVC
svc = SVC(probability=True)

# Creating the model using GridSearchCV with the parameter grid
model = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)

# Training the model using training data
model.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found:", model.best_params_)

# Testing the model using test data
y_pred = model.predict(X_test)

# Printing the classification report
print(classification_report(y_test, y_pred))

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")

# SVM with PCA - sample of dataset

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# Indices for training and testing data as defined in the 'subset' column
train_indices = df[df['subset'] == 'train'].index
test_indices = df[df['subset'] == 'test'].index

# Randomly select 20% of the data from both training and testing subsets
train_indices_sample = train_test_split(train_indices, train_size=0.2, random_state=42)[0]
test_indices_sample = train_test_split(test_indices, test_size=0.2, random_state=42)[1]

# Assign features and targets based on the selected indices
X_train = df_matrix_reduced[train_indices_sample]
X_test = df_matrix_reduced[test_indices_sample]
y_train = df.loc[train_indices_sample, 'category']
y_test = df.loc[test_indices_sample, 'category']

# Import and configure the SVM model with GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.0001, 0.001, 0.1, 1],
    'kernel': ['rbf', 'poly']
}

svc = SVC(probability=True)
model = GridSearchCV(svc, param_grid, cv=5, n_jobs=-1)
model.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found:", model.best_params_)

# Testing the model using test data
y_pred = model.predict(X_test)

# Printing the classification report
print(classification_report(y_test, y_pred))

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")

# SVM without PCA - sample of dataset

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# Indices for training and testing data as defined in the 'subset' column
train_indices = df[df['subset'] == 'train'].index
test_indices = df[df['subset'] == 'test'].index

# Randomly select 20% of the data from both training and testing subsets
train_indices_sample = train_test_split(train_indices, train_size=0.2, random_state=42)[0]
test_indices_sample = train_test_split(test_indices, test_size=0.2, random_state=42)[1]

# Assign features and targets based on the selected indices
X_train = image_matrix_scaled[train_indices_sample]
X_test = image_matrix_scaled[test_indices_sample]
y_train = df.loc[train_indices_sample, 'category']
y_test = df.loc[test_indices_sample, 'category']

# Import and configure the SVM model with GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.0001, 0.001, 0.1, 1],
    'kernel': ['rbf', 'poly']
}

svc = SVC(probability=True)
model = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)
model.fit(X_train, y_train)

# Divide test data into 4 batches for classification
batch_size = len(X_test) // 4  # Calculate the batch size based on the total number of samples in the test set

for i in range(4):
    start_index = i * batch_size
    end_index = start_index + batch_size if i < 3 else len(X_test)  # Ensure the last batch includes all remaining data

    X_batch = X_test[start_index:end_index]
    y_batch = y_test[start_index:end_index]
    
    # Predict model on the batch
    y_pred_batch = model.predict(X_batch)
    
    # Print the classification report for the current batch
    print(f"Classification Report for Batch {i+1}")
    print(classification_report(y_batch, y_pred_batch))
    print("\n" + "-"*80 + "\n")

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")

# SVM with PCA with SGDClassifier 

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# SVM without PCA
X_train = df_matrix_reduced[df['subset'] == 'train']
y_train = df[df['subset'] == 'train']['category']
X_test = df_matrix_reduced[df['subset'] == 'test']
y_test = df[df['subset'] == 'test']['category']

# Define parameter grid for SGDClassifier
param_grid_sgd = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Regularization parameter
    'penalty': ['l2', 'l1', 'elasticnet'],
    'loss': ['hinge']  # Hinge loss corresponds to a linear SVM
}

# Create SGDClassifier
sgd = SGDClassifier(max_iter=1000, tol=1e-3)

# Create GridSearchCV model
grid_search_sgd = GridSearchCV(sgd, param_grid_sgd, cv=3, n_jobs=-1)

# Train model using training data
grid_search_sgd.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found:", grid_search_sgd.best_params_)

# Test model using test data
y_pred_sgd = grid_search_sgd.predict(X_test)

# Print classification report
print(classification_report(y_test,y_pred_sgd))

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")

# SVM without PCA with SGDClassifier- sample of dataset

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# Indices for training and testing data as defined in the 'subset' column
train_indices = df[df['subset'] == 'train'].index
test_indices = df[df['subset'] == 'test'].index

# Randomly select 20% of the data from both training and testing subsets
train_indices_sample = train_test_split(train_indices, train_size=0.2, random_state=42)[0]
test_indices_sample = train_test_split(test_indices, test_size=0.2, random_state=42)[1]

# Assign features and targets based on the selected indices
X_train = image_matrix_scaled[train_indices_sample]
X_test = image_matrix_scaled[test_indices_sample]
y_train = df.loc[train_indices_sample, 'category']
y_test = df.loc[test_indices_sample, 'category']

# Define parameter grid for SGDClassifier
param_grid_sgd = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Regularization parameter
    'penalty': ['l2', 'l1', 'elasticnet'],
    'loss': ['hinge']  # Hinge loss corresponds to a linear SVM
}

# Create SGDClassifier
sgd = SGDClassifier(max_iter=1000, tol=1e-3)

# Create GridSearchCV model
grid_search_sgd = GridSearchCV(sgd, param_grid_sgd, cv=3, n_jobs=-1)

# Train model using training data
grid_search_sgd.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found:", grid_search_sgd.best_params_)

# Divide test data into 4 batches for classification
batch_size = len(X_test) // 4  # Calculate the batch size based on the total number of samples in the test set

for i in range(4):
    start_index = i * batch_size
    end_index = start_index + batch_size if i < 3 else len(X_test)  # Ensure the last batch includes all remaining data

    X_batch = X_test[start_index:end_index]
    y_batch = y_test[start_index:end_index]
    
    # Predict model on the batch
    y_pred_batch = grid_search_sgd.predict(X_batch)
    
    # Print the classification report for the current batch
    print(f"Classification Report for Batch {i+1}")
    print(classification_report(y_batch, y_pred_batch))
    print("\n" + "-"*80 + "\n")

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")


# SVM with PCA with SGDClassifier - sample of a dataset

In [ ]:
# Start timing the overall process
start_time_overall = time.time()

# Indices for training and testing data as defined in the 'subset' column
train_indices = df[df['subset'] == 'train'].index
test_indices = df[df['subset'] == 'test'].index

# Randomly select 20% of the data from both training and testing subsets
train_indices_sample = train_test_split(train_indices, train_size=0.2, random_state=42)[0]
test_indices_sample = train_test_split(test_indices, test_size=0.2, random_state=42)[1]

# Assign features and targets based on the selected indices
X_train = df_matrix_reduced[train_indices_sample]
X_test = df_matrix_reduced[test_indices_sample]
y_train = df.loc[train_indices_sample, 'category']
y_test = df.loc[test_indices_sample, 'category']

# Define parameter grid for SGDClassifier
param_grid_sgd = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Regularization parameter
    'penalty': ['l2', 'l1', 'elasticnet'],
    'loss': ['hinge']  # Hinge loss corresponds to a linear SVM
}

# Create SGDClassifier
sgd = SGDClassifier(max_iter=1000, tol=1e-3)

# Create GridSearchCV model
grid_search_sgd = GridSearchCV(sgd, param_grid_sgd, cv=3, n_jobs=-1)

# Train model using training data
grid_search_sgd.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found:", grid_search_sgd.best_params_)

# Divide test data into 4 batches for classification
batch_size = len(X_test) // 4  # Calculate the batch size based on the total number of samples in the test set

for i in range(4):
    start_index = i * batch_size
    end_index = start_index + batch_size if i < 3 else len(X_test)  # Ensure the last batch includes all remaining data

    X_batch = X_test[start_index:end_index]
    y_batch = y_test[start_index:end_index]
    
    # Predict model on the batch
    y_pred_batch = grid_search_sgd.predict(X_batch)
    
    # Print the classification report for the current batch
    print(f"Classification Report for Batch {i+1}")
    print(classification_report(y_batch, y_pred_batch))
    print("\n" + "-"*80 + "\n")

# Calculate and print the overall running time
overall_time = time.time() - start_time_overall
print(f"Total running time: {overall_time:.2f} seconds.")

# Visualising the results of SVM

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Model instantiation
modelSVC = SVC()

# Set train_sizes values to start with a larger subset to ensure each split has sufficient data
train_sizes = np.linspace(0.4, 1.0, 5)  # Start at 40% to avoid too small training sets

# Use stratified cross-validation
cv = StratifiedKFold(n_splits=3)

# Compute learning curve
try:
    train_sizes, train_scores, val_scores = learning_curve(
        modelSVC, X_train, y_train, cv=3, n_jobs=-1, train_sizes=train_sizes, error_score='raise'
    )
except ValueError as e:
    print("Error occurred during learning curve computation:")
    print(e)

# Compute mean scores if learning curve computation is successful
if 'train_sizes' in locals() and 'train_scores' in locals() and 'val_scores' in locals():
    train_scores_mean = np.mean(train_scores, axis=1)
    val_scores_mean = np.mean(val_scores, axis=1)

    # Plot learning curve
    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, val_scores_mean, label='Cross-validation score')
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    plt.title('Learning Curves')
    plt.legend(loc='best')
    plt.show()
else:
    print("Learning curve computation failed due to insufficient class representation in some splits.")

# CNN - TF & Keras (Version 1)

LOAD Dataset

In [ ]:
df['category_encoded'] = df['category'].apply(lambda x: to_categorical(x, num_classes=3))


Split the data into training, validation, and test sets

In [ ]:
train_df, test_df = train_test_split(df[df['subset'] == 'train'], test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)  # This will make 60% train, 20% validation


Function to convert DataFrame to tf.data.Dataset with optional augmentation

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32, augment=False):
    images = np.stack(dataframe['image_array'].values)
    labels = np.stack(dataframe['category_encoded'].values)
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    if augment:
        dataset = dataset.map(lambda img, lbl: (tf.image.random_flip_left_right(img), lbl),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

Create TensorFlow datasets

In [ ]:
train_ds = df_to_dataset(train_df, shuffle=True, batch_size=32, augment=True)
val_ds = df_to_dataset(val_df, batch_size=32, shuffle=False)
test_ds = df_to_dataset(test_df, batch_size=32, shuffle=False)


Define the CNN model 

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Train the model

history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds
)

Evaluate the model on the test dataset

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)

Plot training and validation accuracy and loss

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

Classification Report and Confusion Matrix

In [ ]:
# Predict the labels on the test dataset
test_images, test_labels = next(iter(test_ds.unbatch().batch(len(test_df))))
predicted_probs = model.predict(test_images)
predicted_labels = np.argmax(predicted_probs, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Generate the classification report
class_report = classification_report(true_labels, predicted_labels, target_names=['Class 0', 'Class 1', 'Class 2'])
print("Classification Report:")
print(class_report)

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


# CNN  - TF&Keras (Version 2)

Load Dataset

In [ ]:
df['category_encoded'] = df['category'].apply(lambda x: to_categorical(x, num_classes=3))

Split the data into training, validation, and test sets

In [ ]:
train_df, test_df = train_test_split(df[df['subset'] == 'train'], test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)  # This will make 60% train, 20% validation

Function to convert DataFrame to tf.data.Dataset with optional augmentation

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32, augment=False):
    images = np.stack(dataframe['image_array'].values)
    labels = np.stack(dataframe['category_encoded'].values)
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    if augment:
        dataset = dataset.map(lambda img, lbl: (tf.image.random_flip_left_right(img), lbl),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

Create TensorFlow Datasets

In [ ]:
train_ds = df_to_dataset(train_df, shuffle=True, batch_size=32, augment=True)
val_ds = df_to_dataset(val_df, batch_size=32, shuffle=False)
test_ds = df_to_dataset(test_df, batch_size=32, shuffle=False)


Define the Model Building Function

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=32),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
                     activation='relu',
                     input_shape=(128, 128, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))

    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(Conv2D(filters=hp.Int(f'conv_{i+2}_filter', min_value=32, max_value=128, step=32),
                         kernel_size=hp.Choice(f'conv_{i+2}_kernel', values=[3,5]),
                         activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2, 2))

    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_units', min_value=512, max_value=1024, step=256),
                    activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

Set Up the Hyperparameter Search

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of variations on model
    executions_per_trial=1,  # Number of models that should be built and fit for each trial
    directory='output',
    project_name='ModelTuning'
)

# Display search space summary
tuner.search_space_summary()

Start the Hyperparameter Tuning

In [ ]:
tuner.search(train_ds, epochs=10, validation_data=val_ds)

Get the Best Model

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Evaluate the best model

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Predict and Report

In [ ]:
test_images, test_labels = next(iter(test_ds.unbatch().batch(len(test_df))))
predicted_probs = best_model.predict(test_images)
predicted_labels = np.argmax(predicted_probs, axis=1)
true_labels = np.argmax(test_labels, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=['Class 0', 'Class 1', 'Class 2']))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))


In [ ]:
# Running time histogram